In [ ]:
'''
Landsat 5 (years with images):
  1990: 2 images
  1991: 3 images
  1992: 1 images
  1993: 3 images
  1994: 3 images
  1995: 2 images
  1996: 1 images
  1997: 1 images
  1998: 1 images
  1999: 2 images
  2000: 1 images
  2001: 1 images
  2002: 1 images
  2003: 1 images
  2005: 2 images
  2006: 2 images
  2007: 1 images
  2008: 2 images
  2010: 2 images
  2011: 3 images
Landsat 8 (years with images):
  2013: 1 images
  2014: 1 images
  2015: 3 images
  2016: 1 images
  2017: 2 images
  2018: 2 images
  2022: 1 images
  2023: 1 images
  2024: 1 images
'''

In [1]:
import ee
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
import json
import sys
sys.path.append('../Setup')  # Go up one level to reach Setup folder
from gee_setup import *

setup_gee()

✅ earthengine-api imported successfully
✅ folium imported successfully
✅ pandas imported successfully
✅ numpy imported successfully
✅ matplotlib imported successfully
✅ seaborn imported successfully
✅ IPython display imported successfully
✅ json imported successfully (built-in)
GEE utilities loaded successfully!
Use quick_setup() for one-line initialization
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3


True

In [23]:
# Import analysis functions
sys.path.append('../functions')
from geolib import (
    count_images_by_year, 
    image_count, 
    count_images_by_week,
    mask_clouds_landsat,
    ndsi_l5,
    ndsi_l9
)

In [15]:
aoi = ee.Geometry.Polygon([[
    [-106.99065650552905, 39.088354116315266],
    [-106.71050513834155, 38.652122067998306], 
    [-106.14470923990405, 38.596331654032184],
    [-106.40838111490405, 39.21189167999414],
    [-106.99065650552905, 39.088354116315266]
]])
print("AOI defined - High Rocky Mountains polygon")
print(f"AOI area: {aoi.area().divide(1000000).getInfo():.2f} square kilometers")

date_range_1 = ee.DateRange(['1990-01-01', '2000-01-01'])  # Landsat 5 period
date_range_2 = ee.DateRange(['2015-01-01', '2025-01-01'])  # Landsat 8/9 period

AOI defined - High Rocky Mountains polygon
AOI area: 2994.93 square kilometers


In [18]:
# Fix the date ranges and add missing cloud filter
aoi = ee.Geometry.Polygon([[
    [-106.99065650552905, 39.088354116315266],
    [-106.71050513834155, 38.652122067998306], 
    [-106.14470923990405, 38.596331654032184],
    [-106.40838111490405, 39.21189167999414],
    [-106.99065650552905, 39.088354116315266]
]])

print("AOI defined - High Rocky Mountains polygon")
print(f"AOI area: {aoi.area().divide(1000000).getInfo():.2f} square kilometers")

# Fixed date ranges (remove square brackets)
date_range_1 = ee.DateRange('1990-01-01', '2000-01-01')  # Landsat 5 period
date_range_2 = ee.DateRange('2015-01-01', '2025-01-01')  # Landsat 8/9 period

# Create collections
landsat_1 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterDate(date_range_1) \
    .filterBounds(aoi) \
    .filter(ee.Filter.calendarRange(6, 6, 'month')) \
    .filter(ee.Filter.lt('CLOUD_COVER', 10))

landsat_2 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(date_range_2) \
    .filterBounds(aoi) \
    .filter(ee.Filter.calendarRange(6, 6, 'month')) \
    .filter(ee.Filter.lt('CLOUD_COVER', 10))

test = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(ee.DateRange('1980-01-01', '2025-01-01')) \
    .filterBounds(aoi) \
    .filter(ee.Filter.calendarRange(6, 6, 'month')) \
    .filter(ee.Filter.lt('CLOUD_COVER', 10))

collection = landsat_1.merge(landsat_2)

print("✅ Collections created successfully")

AOI defined - High Rocky Mountains polygon
AOI area: 2994.93 square kilometers
✅ Collections created successfully


In [ ]:
# Test image counting functions
print("=== JUNE IMAGES BY YEAR ===")
year_results = count_images_by_year(landsat_2).getInfo()

# Display results in a clean format
for feature in year_results['features']:
    year = feature['properties']['year']
    count = feature['properties']['image_count']
    if count > 0:
        print(f"{year}: {count} images")

print(f"\n=== SUMMARY ===")
total_images = test.size().getInfo()
print(f"Total June images (all years): {total_images}")
print("✅ Function imports working correctly")

=== JUNE IMAGES BY YEAR ===
2015: 3 images
2016: 1 images
2017: 2 images
2018: 2 images
2022: 1 images
2023: 1 images
2024: 1 images

=== SUMMARY ===
Total June images (all years): 13
✅ Function imports working correctly
2015: 3 images
2016: 1 images
2017: 2 images
2018: 2 images
2022: 1 images
2023: 1 images
2024: 1 images

=== SUMMARY ===
Total June images (all years): 13
✅ Function imports working correctly


In [29]:
# Direct test - manually check specific years
print("=== MANUAL YEAR CHECK FOR LANDSAT_1 ===")

# Test specific years where we know there should be data
test_years = [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999]

for year in test_years:
    # Create year-specific collection
    year_start = ee.Date.fromYMD(year, 1, 1)
    year_end = year_start.advance(1, 'year')
    
    year_collection = landsat_1.filterDate(year_start, year_end)
    count = year_collection.size().getInfo()
    
    if count > 0:
        print(f"{year}: {count} images")

print(f"\n=== VERIFICATION: Check if any images exist at all ===")
if landsat_1.size().getInfo() > 0:
    # Get first image info
    first = landsat_1.first()
    first_date = ee.Date(first.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    print(f"First image date: {first_date}")
    
    # Get date range of all images
    dates = landsat_1.aggregate_array('system:time_start')
    date_list = dates.getInfo()
    if date_list:
        import datetime
        readable_dates = []
        for timestamp in date_list[:5]:  # Show first 5 dates
            dt = datetime.datetime.fromtimestamp(timestamp/1000)
            readable_dates.append(dt.strftime('%Y-%m-%d'))
        print(f"Sample dates: {readable_dates}")

print("✅ Manual verification complete")

=== MANUAL YEAR CHECK FOR LANDSAT_1 ===
1990: 2 images
1991: 3 images
1992: 1 images
1991: 3 images
1992: 1 images
1993: 3 images
1993: 3 images
1994: 3 images
1995: 2 images
1994: 3 images
1995: 2 images
1996: 1 images
1997: 1 images
1996: 1 images
1997: 1 images
1998: 1 images
1999: 2 images

=== VERIFICATION: Check if any images exist at all ===
1998: 1 images
1999: 2 images

=== VERIFICATION: Check if any images exist at all ===
First image date: 1991-06-17
First image date: 1991-06-17
Sample dates: ['1991-06-17', '1992-06-19', '1993-06-22', '1994-06-09', '1994-06-25']
✅ Manual verification complete
Sample dates: ['1991-06-17', '1992-06-19', '1993-06-22', '1994-06-09', '1994-06-25']
✅ Manual verification complete


In [32]:
## create cloud mask on each image
landsat_1_masked = landsat_1.map(mask_clouds_landsat)
landsat_2_masked = landsat_2.map(mask_clouds_landsat)


In [33]:
## apply ndvi to each image
landsat_1_ndsi = landsat_1_masked.map(ndsi_l5)
landsat_2_ndsi = landsat_2_masked.map(ndsi_l9)

In [ ]:
## create a weighted average by year composite image from each collection
